### Install any required libraries

In [2]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

In [3]:
!pip install -qU faiss_cpu pymupdf pandas

In [4]:
!pip install -qU ragas

### Set environment variables

In [5]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

#### Load Data

In [6]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "input_data/nvidia_10k.pdf",
)

documents = loader.load()

In [7]:
documents[0].metadata

{'source': 'input_data/nvidia_10k.pdf',
 'file_path': 'input_data/nvidia_10k.pdf',
 'page': 0,
 'total_pages': 96,
 'format': 'PDF 1.4',
 'title': '0001045810-24-000029',
 'author': 'EDGAR® Online LLC, a subsidiary of OTC Markets Group',
 'subject': 'Form 10-K filed on 2024-02-21 for the period ending 2024-01-28',
 'keywords': '0001045810-24-000029; ; 10-K',
 'creator': 'EDGAR Filing HTML Converter',
 'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0',
 'creationDate': "D:20240221173732-05'00'",
 'modDate': "D:20240221173744-05'00'",
 'trapped': '',
 'encryption': 'Standard V2 R3 128-bit RC4'}

#### Transform - Split documents into chunks

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

In [10]:
len(documents)

624

#### Load OpenAI Embeddings model

In [14]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

#### Create a FAISS Vector Store

In [15]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(documents, embeddings)

#### Create a Retriever

In [17]:
retriever = vector_store.as_retriever()
retrieved_documents = retriever.invoke("Who is the E-VP, Operations - and how old are they?")

In [18]:
for doc in retrieved_documents:
  print(doc)

page_content="Debora Shoquist\n69\nExecutive Vice President, Operations\nTimothy S. Teter\n57\nExecutive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since our\ninception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductor\ncompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University." metadata={'source': 'input_data/nvidia_10k.pdf', 'file_path': 'input_data/nvidia_10k.pdf', 'page': 11, 'total_pages': 96, 'format': 'PDF 1.4', 'title': '0001045810-24-000029', 'author': 'EDGAR® Online LLC, a subsidiary of OTC Markets Group', 'subject': 'Form 10-K filed on 202

#### Create a RAG Chain

In [19]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [20]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

#### Mid-Term Challenge Tests

#### Who is the E-VP, Operations - and how old are they?

In [21]:
question = "Who is the E-VP, Operations - and how old are they?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Debora Shoquist, 69 years old.


#### What is the gross carrying amount of Total Amortizable Intangible Assets for Jan 29, 2023?

In [22]:
question = "What is the gross carrying amount of Total Amortizable Intangible Assets for Jan 29, 2023?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

$3,539 million


#### Evaluation using RAGAS

In [25]:
eval_documents = documents

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 400
)

eval_documents = text_splitter.split_documents(eval_documents)

In [26]:
len(documents)

624

In [27]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.with_openai()

testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.25, reasoning: 0.25, multi_context: 0.5})

/var/folders/_1/hpp7wgls4pb3zx13lgn3cp8w0000gn/T/ipykernel_93059/735169035.py:4: DeprecationWarning: The function with_openai was deprecated in 0.1.4, and will be removed in the 0.2.0 release. Use from_langchain instead.
  generator = TestsetGenerator.with_openai()


embedding nodes:   0%|          | 0/1248 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/10 [00:00<?, ?it/s]

In [28]:
testset.test_data[0]

DataRow(question='What steps does the vendor risk assessment process involve to mitigate cybersecurity threats?', contexts=['mitigate and recover from identified and significant cybersecurity threats.\nWe also have a vendor risk assessment process consisting of the distribution and review of supplier questionnaires designed to help us evaluate cybersecurity\nrisks that we may encounter when working with third parties that have access to confidential and other sensitive company information. We take steps designed\nto ensure that such vendors have implemented data privacy and security controls that help mitigate the cybersecurity risks associated with these vendors. We'], ground_truth='The vendor risk assessment process involves the distribution and review of supplier questionnaires designed to evaluate cybersecurity risks. The goal is to ensure that vendors have implemented data privacy and security controls to mitigate cybersecurity risks.', evolution_type='simple', metadata=[{'source'

In [30]:
test_df = testset.to_pandas()
test_df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What steps does the vendor risk assessment pro...,[mitigate and recover from identified and sign...,The vendor risk assessment process involves th...,simple,"[{'source': 'input_data/nvidia_10k.pdf', 'file...",True
1,How has NVIDIA been successful in attracting g...,[relating to human capital management.\nTo be ...,NVIDIA has been successful in attracting globa...,simple,"[{'source': 'input_data/nvidia_10k.pdf', 'file...",True
2,How is revenue designated based on billing loc...,[Table of Contents\nAll Other operating loss -...,Revenue by geographic region is designated bas...,reasoning,"[{'source': 'input_data/nvidia_10k.pdf', 'file...",True
3,How did the U.S. federal research tax credit a...,[The effective tax rate increased due to a dec...,"The U.S. federal research tax credit, along wi...",reasoning,"[{'source': 'input_data/nvidia_10k.pdf', 'file...",True
4,What is the purpose of a company's internal co...,[procedures as we considered necessary in the ...,A company's internal control over financial re...,multi_context,"[{'source': 'input_data/nvidia_10k.pdf', 'file...",True


In [31]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

In [32]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

In [33]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [34]:
response_dataset[0]

{'question': 'What steps does the vendor risk assessment process involve to mitigate cybersecurity threats?',
 'answer': 'The vendor risk assessment process involves the distribution and review of supplier questionnaires designed to evaluate cybersecurity risks associated with third parties.',
 'contexts': ['mitigate and recover from identified and significant cybersecurity threats.\nWe also have a vendor risk assessment process consisting of the distribution and review of supplier questionnaires designed to help us evaluate cybersecurity\nrisks that we may encounter when working with third parties that have access to confidential and other sensitive company information. We take steps designed\nto ensure that such vendors have implemented data privacy and security controls that help mitigate the cybersecurity risks associated with these vendors. We',
  'routinely assess our high-risk suppliers’ conformance to industry standards (e.g., ISO 27001, ISO 28001, and C-TPAT), and we evaluate 

In [35]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [36]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

#### RAGAS Results

In [37]:
results

{'faithfulness': 0.9500, 'answer_relevancy': 0.9488, 'context_recall': 1.0000, 'context_precision': 0.7861, 'answer_correctness': 0.7919}

In [38]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What steps does the vendor risk assessment pro...,The vendor risk assessment process involves th...,[mitigate and recover from identified and sign...,The vendor risk assessment process involves th...,1.0,0.941041,1.0,0.833333,0.620211
1,How has NVIDIA been successful in attracting g...,NVIDIA has been successful in attracting globa...,[relating to human capital management.\nTo be ...,NVIDIA has been successful in attracting globa...,1.0,0.999999,1.0,0.833333,1.000000
2,How is revenue designated based on billing loc...,Revenue is designated based on the billing loc...,"[10 \n(2)\n— \nTotal\n$\n(4,890)\n$\n(5,411)\n...",Revenue by geographic region is designated bas...,1.0,0.900000,1.0,1.000000,0.743623
3,How did the U.S. federal research tax credit a...,"The U.S. federal research tax credit, along wi...",[The effective tax rate increased due to a dec...,"The U.S. federal research tax credit, along wi...",0.5,0.854537,1.0,0.916667,0.745303
4,What is the purpose of a company's internal co...,The purpose of a company's internal control ov...,[Management’s Annual Report on Internal Contro...,A company's internal control over financial re...,1.0,0.950108,1.0,1.000000,0.743341
5,How does NVIDIA align their principles with su...,NVIDIA aligns their principles with sustainabi...,[Sustainability and Governance\nNVIDIA invents...,"NVIDIA integrates sound environmental, social,...",1.0,0.965690,1.0,1.000000,0.838499
6,What technologies does NVIDIA RTX use to enhan...,NVIDIA RTX uses ray tracing technology and dee...,[television graphics.\nThe NVIDIA RTX platform...,NVIDIA RTX uses ray tracing technology and dee...,1.0,0.976349,1.0,0.638889,0.999318
7,What types of stock-based awards can be grante...,The types of stock-based awards that can be gr...,[The 2007 Plan authorizes the issuance of ince...,The types of stock-based awards that can be gr...,1.0,0.955835,1.0,0.805556,0.617505
8,What is the purpose of rebates for resellers a...,The purpose of rebates for resellers is to ser...,"[For products sold with a right of return, we ...",The purpose of rebates for resellers is to ser...,1.0,0.944586,1.0,0.000000,0.611390
9,How has NVIDIA been successful in attracting g...,NVIDIA has been successful in attracting globa...,[relating to human capital management.\nTo be ...,NVIDIA has been successful in attracting globa...,1.0,1.000000,1.0,0.833333,1.000000
